In [2]:
from collections import deque
from nltk.util import ngrams
def get_ngrams(sequence, n):
    """
    COMPLETE THIS FUNCTION (PART 1)
    Given a sequence, this function should return a list of n-grams, where each n-gram is a Python tuple.
    This should work for arbitrary values of 1 <= n < len(sequence).
    """
    return list(ngrams(tokenize,n))
    
    sequence = deque(sequence)
    if n>1:
        sequence.extendleft(['START']*(n-1))
    else:
        sequence.extendleft(['START'])
    sequence.extend(['STOP'])
    sequence = list(sequence)
    ngrams = []
    for seq in range(0,len(sequence)-(n-1)):
        ngrams.append(tuple(sequence[seq:seq+n]))
#     print(ngrams)
    return ngrams

In [3]:
from collections import Counter
def count_ngrams(corpus):
    """
    COMPLETE THIS METHOD (PART 2)
    Given a corpus iterator, populate dictionaries of unigram, bigram,
    and trigram counts. 
    """
    unigramcounts = Counter(get_ngrams(corpus,1))
    bigramcounts = Counter(get_ngrams(corpus,2))
    trigramcounts = Counter(get_ngrams(corpus,3))
    total_words = sum(unigramcounts.values())
    return unigramcounts, bigramcounts, trigramcounts, total_words
    return

In [4]:
def raw_trigram_probability(self,trigram):
    """
    COMPLETE THIS METHOD (PART 3)
    Returns the raw (unsmoothed) trigram probability
    """
    return round(trigramcounts[trigram]/bigramcounts[trigram[0:2]],2)

def raw_bigram_probability(self, bigram):
    """
    COMPLETE THIS METHOD (PART 3)
    Returns the raw (unsmoothed) bigram probability
    """
    return round(bigramcounts[bigram]/unigramcounts[bigram[0]],2) #no backing off to unigram probability

def raw_unigram_probability(self, unigram):
    """
    COMPLETE THIS METHOD (PART 3)
    Returns the raw (unsmoothed) unigram probability.
    """
    #hint: recomputing the denominator every time the method is called
    # can be slow! You might want to compute the total number of words once, 
    # store in the TrigramModel instance, and then re-use it.  
    return round(unigramcounts[unigram]/total_words,2)

In [6]:
def smoothed_trigram_probability(self, trigram):
    """
    COMPLETE THIS METHOD (PART 4)
    Returns the smoothed trigram probability (using linear interpolation). 
    """
    lambda1 = 1/3.0
    lambda2 = 1/3.0
    lambda3 = 1/3.0
    return lambda1 * raw_trigram_probability(trigram) + lambda2 * raw_bigram_probability(trigram[1:]) + lambda3 * raw_unigram_probability(trigram[2])
    return 0.0

In [7]:
import math
def sentence_logprob(sentence):
    trigram = get_ngrams(sentence,3)
    sent_prob = 0
    for tri in trigram:
        sent_prob+= math.log2(smoothed_trigram_probability(tri))
    return sent_prob

In [8]:
def perplexity(corpus):
    sent_logprob = 0
    for sentence in corpus:
        sent_logprob+=math.log2(sentence_logprob(sentence))
    norm_sent_logprob = sent_logprob/total_words
    return 2**(-norm_sent_logprob)

In [9]:
if __name__=='__main__':
    s = ["natural","language","processing","language","processed","natural","natural","language","processing"]
    u = get_ngrams(s,1)
    print("========================")
    b = get_ngrams(s,2)
    print("========================")
    t = get_ngrams(s,3)
    print("\n\n")
    unigramcounts, bigramcounts, trigramcounts, total_words = count_ngrams(s)
    print(t,"\n")
    print(trigramcounts)
    print(trigramcounts[t[0]])

UnboundLocalError: local variable 'ngrams' referenced before assignment

In [13]:
from collections import deque as dq
l=["natural","language","processing"]
c = dq(l)
c
c.extendleft(['START']*2)
c.extend(['STOP'])
c = list(c)
for i in range(0,len(c)-2):
    print(tuple(c[i:i+3]), i)

('START', 'START', 'natural') 0
('START', 'natural', 'language') 1
('natural', 'language', 'processing') 2
('language', 'processing', 'STOP') 3


In [39]:
from collections import Counter
cnt = Counter([('START','START','the')])
cnt[('START','START','the')]

1

In [41]:
cnt['hello']

0

In [ ]:
## This file assumes Python 3
## To work with Python 2, you would need to adjust
## at least: the print statements (remove parentheses)
## and the instances of division (convert
## arguments of / to floats), and possibly other things
## -- I have not tested this.

import nltk
from nltk.corpus import brown

test_sentence_tokens = ['a','fact','about','the','unicorn','is','the','same','as','an','alternative','fact','about','the','unicorn','.']

words = brown.words()
fdist1 = nltk.FreqDist(w.lower() for w in words)

total_words = len(words)

print('Frequency of tokens in sample sententence in Brown according to NLTK:')

for word in test_sentence_tokens:
    print(word,fdist1[word])

input('Pausing: Hit Return when Ready.')

print('Given that there are',total_words,'in the Brown Corpus, the unigram probability of these words')
print('is as follows (rounded to 3 significant digits):')

for word in test_sentence_tokens:
    unigram_probability = fdist1[word]/total_words
    print(word,float('%.3g' % unigram_probability))
    ## print(word,round((fdist1[word]/total_words),3))

input('Pausing: Hit Return when Ready.')

## ADD convert single count items to OOV
## make simple assumption about sentence endings,
## and the position of START and END (sentence boundaries)

words2 = []
previous = 'EMPTY'
sentences = 0
for word in words:
    if previous in ['EMPTY','.','?','!']:
        ## insert word_boundaries at beginning of Brown,
        ## and after end-of-sentence markers (overgenerate due to abbreviations, etc.)
        words2.append('*start_end*')
    if fdist1[word]==1:
        ## words occurring only once are treated as Out of Vocabulary Words
        words2.append('*oov*')
    else:
        words2.append(word)
    previous = word
words2.append('*start_end*') ## assume one additional *start_end* at the end of Brown

fdist2 = nltk.FreqDist(w.lower() for w in words2)
## get Unigram counts for all words occuring more than once
## and also a count for OOV words

print('There are',fdist2['*oov*'],'instances of OOVs')

print('Unigram probabilities including OOV probabilities.')

def get_unigram_probability(word):
    if word in fdist1:
        unigram_probability = fdist2[word]/total_words
    else:
        unigram_probability = fdist2['*oov*']/total_words
    return(unigram_probability)

for word in test_sentence_tokens:
    unigram_probability = get_unigram_probability(word)
    print(word,float('%.3g' % unigram_probability))

input('Pausing: Hit Return when Ready.')
## make new version that models Out of Vocabulary (OOV) words

print('Calculating bigram counts for sentence, including bigrams with sentence boundaries, i.e., *BEGIN* and *END*')
print('Assuming some idealizations: all periods, questions and exclamation marks end sentences;')

bigrams = nltk.bigrams(w.lower() for w in words2)
## get bigrams for words2 (words plus OOV)

cfd = nltk.ConditionalFreqDist(bigrams)

# for token1 in cfd:
#     if not '*oov*' in cfd[token1]:
#         cfd[token1]['*oov*']=1
#         ## fudge so there can be no 
#         ## 0 bigram

def multiply_list(inlist):
    out = 1
    for number in inlist:
        out *= number
    return(out)

def get_bigram_probability(first,second):
    if not second in cfd[first]:
        print('Backing Off to Unigram Probability for',second)
        unigram_probability = get_unigram_probability(second)
        return(unigram_probability)
    else:
        bigram_frequency = cfd[first][second]
    unigram_frequency = fdist2[first]
    bigram_probability = bigram_frequency/unigram_frequency
    return(bigram_probability)

def calculate_bigram_freq_of_sentence_token_list(tokens):
    prob_list = []
    ## assume that 'START' precedes the first token
    previous = '*start_end*'
    for token in tokens:
        if not token  in fdist2:
            token = '*oov*'
        next_probability = get_bigram_probability(previous,token)
        print(previous,token,(float('%.3g' % next_probability)))
        prob_list.append(next_probability)
        previous = token
    ## assume that 'END' follows the last token
    next_probability = get_bigram_probability(previous,'*start_end*')
    print(previous,'*start_end*',next_probability)
    prob_list.append(next_probability)
    probability = multiply_list(prob_list)
    print('Total Probability',float('%.3g' % probability))
    return(probability)



result = calculate_bigram_freq_of_sentence_token_list(test_sentence_tokens)

In [ ]:
import nltk

In [ ]:
nltk.download('brown')

In [16]:
sum(Counter([1,2,3,4,5,1,2,1,6]).values())

9

In [46]:
round(0.394551,1)
2/4

0.5

In [49]:
if('h' in 'hello'):
    print('yes')

yes


In [50]:
l=[1,2,3,4]
l[1:2]

[2]

In [56]:
s=(1,2,3)
s[1:3]

(2, 3)

In [57]:
import math
math.log2(2)

1.0

In [63]:
2**-4

0.0625

In [2]:
from collections import Counter
cnt = Counter(['a','b','c','a'])
cnt2 = Counter(['a'])
cnt3 = Counter()
cnt3+=cnt
cnt3+=cnt2
cnt3

3